In [99]:
#First Time Setup
!pip install pandas
!pip install numpy
!pip install google
!pip install spacey
!python3 -m spacy download xx_ent_wiki_sm
!pip install python-dotenv

  Using cached https://github.com/explosion/spacy-models/releases/download/xx_ent_wiki_sm-3.0.0/xx_ent_wiki_sm-3.0.0-py3-none-any.whl (15.6 MB)
✔ Download and installation successful
You can now load the package via spacy.load('xx_ent_wiki_sm')


In [ ]:
import pandas as pd
from itertools import combinations
from googlesearch import search
import spacy
import requests
from dotenv import load_dotenv
load_dotenv()
import os

# Create all possible Interrelations

In [ ]:
sheets = ("../../data/Example_Data/companies.csv", "../../data/Example_Data/companies.csv")
dfs = []

for sheet in sheets:
    df = pd.read_csv(sheet)
    dfs.append(df)
    
df = pd.concat(dfs, axis=0, join='inner', ignore_index=False)

# importing product
from itertools import product
  
# apply product method
pairs  = list(product(df['name'], df['url']))

print("%s %s %s %s %s" %("created",len(pairs),"possible combinations from", len(dfs[0].index), "rows"))

# Create all search queries to execute

In [ ]:
connectors = "partner OR collaboration OR partnership OR together OR joint-venture OR plattform OR operates OR colloborates OR signed OR announced"
queries = []

for pair in pairs:
    query = "%s%s%s %s %s%s%s %s%s%s" %('"',pair[0],'"','AND', "(" , connectors,")", 'site:*."',pair[1],'"')
    queries.append(query)

print("%s %s %s" %("created",len(queries),"search queries"))
with open('../../data/queries.txt', 'w') as f:
    for query in queries:
        f.write("%s\n" % query)

# Make a Google Custom Search Engine

Go to https://www.google.com/cse/

Create a custom search engine

Get search engine ID

You can test the custom search engine by directly visiting its "Public URL" which is found in "Setup" => "Details" => "Public URL"
# Make a Google Cloud Console "Project" to get API key

Go to https://console.developers.google.com/apis/dashboard

Create a project or select

Enable APIs and services

Select Custom Search API and enable it

Click "Create credentials"

Get your credentials for Custom Search API

Go to Credentials and get API key
CustomSearch API Client Library for Python

# Create a .env with your GOOGLE_API_KEY an SEARCH_ENGINE_ID (cx)

In [63]:
GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
SEARCH_ENGINE_ID = os.environ.get("SEARCH_ENGINE_ID")

In [87]:
limit = 30
results = []

for query in queries[:limit]:
    try:
        print (query)
        # using the first page
        page = 1
        # constructing the URL
        # doc: https://developers.google.com/custom-search/v1/using_rest
        # calculating start, (page=2) => (start=11), (page=3) => (start=21)
        start = (page - 1) * 10 + 1
        url = f"https://www.googleapis.com/customsearch/v1?key={GOOGLE_API_KEY}&cx={SEARCH_ENGINE_ID}&q={query}&start={start}"
        data = requests.get(url).json()
        # get the result items
        search_items = data.get("items")
        # iterate over 10 results found
        for i, search_item in enumerate(search_items, start=1):
            # get the page title
            title = search_item.get("title")
            # page snippet
            snippet = search_item.get("snippet")
            # alternatively, you can get the HTML snippet (bolded keywords)
            html_snippet = search_item.get("htmlSnippet")
            # extract the page url
            link = search_item.get("link")
            # print the results
#             print("="*10, f"Result #{i+start-1}", "="*10)
#             print("Title:", title)
#             print("Description:", snippet)
#             print("URL:", link, "\n")
            r = {"title": title,
                "snippet": snippet,
                "link": link}
            results.append(r)
            
    except Exception as e:
            print(e)

results
df = pd.DataFrame(results)
df

"Huawei" AND (partner OR collaboration OR partnership OR together OR joint-venture OR plattform OR operates OR colloborates OR signed OR announced) site:*."huawei.com"
"Huawei" AND (partner OR collaboration OR partnership OR together OR joint-venture OR plattform OR operates OR colloborates OR signed OR announced) site:*."baesystems.com"
"Huawei" AND (partner OR collaboration OR partnership OR together OR joint-venture OR plattform OR operates OR colloborates OR signed OR announced) site:*."axis.com"
"Huawei" AND (partner OR collaboration OR partnership OR together OR joint-venture OR plattform OR operates OR colloborates OR signed OR announced) site:*."boschsecurity.com"
'NoneType' object is not iterable
"Huawei" AND (partner OR collaboration OR partnership OR together OR joint-venture OR plattform OR operates OR colloborates OR signed OR announced) site:*."dahuasecurity.com"
"Huawei" AND (partner OR collaboration OR partnership OR together OR joint-venture OR plattform OR operates OR

,title,snippet,link
0,Huawei,Openness . Collaboration . Shared Success. Let...,https://partner.huawei.com/
1,Huawei Intelligent Collaboration Solution — Hu...,Huawei CloudLink Video Conferencing Solution c...,https://e.huawei.com/en/solutions/enterprise-c...
2,"Openness, Collaboration, and Shared Success - ...",Huawei is committed to dissolving boundaries a...,https://www.huawei.com/en/corporate-informatio...
3,Together 2020 - HUAWEI Global,This is the start of an extraordinary new real...,https://consumer.huawei.com/en/campaign/togeth...
4,Become a Partner — Huawei Enterprise,Huawei Partner Program Signup Process. Huawei'...,https://e.huawei.com/us/partner/us/become-a-pa...
...,...,...,...
116,User's Manual,"Jul 5, 2018 ... could void the user's authorit...",https://www.dahuasecurity.com/asset/upload/pro...
117,DIGITAL VIDEO RECORDER,Operate the device within the rated range of p...,https://us.dahuasecurity.com/wp-content/upload...
118,Corporate News,"Aug 2, 2019 ... Nowadays, cameras often work w...",https://www.dahuasecurity.com/newsEvents/press...
119,Actievoorwaarden Dahua Benelux Christmas Promo...,19 nov 2020 ... De Algemene Voorwaarden gelden...,https://www.dahuasecurity.com/asset/upload/upl...


In [89]:
import time
timestr = time.strftime("%Y%m%d-%H%M")
filename = "%s%s%s" %("../../data/result_",timestr,".csv")
df.to_csv(filename, index = False)

In [104]:
# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("xx_ent_wiki_sm")

# Process whole documents
text = df['snippet'][118]
doc = nlp(text)

# Analyze syntax
# print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
# print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

# Find named entities, phrases and concepts
for entity in doc.ents:
    print(entity.text, entity.label_)
    
for ent in doc.ents:
    if ent.label_ == 'DATE':
        print(ent.text) 


Aug MISC
HUAWEI MISC
Alibaba LOC
Sensetime LOC
Megvii PER
